In [64]:
from bs4 import BeautifulSoup
import requests
import copy
import re
import pickle
from collections import defaultdict
import sys
sys.setrecursionlimit(40000)

In [2]:
html_page = requests.get("https://www.the-numbers.com/bankability") #Make a get request to retrieve the page
soup = BeautifulSoup(html_page.content, 'html.parser') #Pass the page contents to beautiful soup for parsing
soup.prettify # test-print

<bound method Tag.prettify of <!DOCTYPE html>

<html>
<head>
<!-- Global site tag (gtag.js) - Google Analytics -->
<script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-1343128-1"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-1343128-1');
</script>
<meta content='(PICS-1.1 "https://www.icra.org/ratingsv02.html" l gen true for "https://www.the-numbers.com/" r (cb 1 lz 1 nz 1 oz 1 vz 1) "https://www.rsac.org/ratingsv01.html" l gen true for "https://www.the-numbers.com/" r (n 0 s 0 v 0 l 0))' http-equiv="PICS-Label"/>
<!--<meta http-equiv="Content-Type" content="text/html; charset=ISO-8859-1" >-->
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="telephone=no" name="format-detection"/> <!-- for apple mobile -->
<meta content="521546213" property="fb:admins">
<meta content="initial-scale=1" name="viewport"/>
<meta content="A monthl

In [3]:
# actors list________________________________________________

dirty_actors_list = list(soup.findAll('span', style="font-size:200%;"))
clean_actors_list = []
for line in dirty_actors_list:
    clean_actors_list.append(line.a.string)
clean_actors_list # test-print

['Tom Cruise',
 'Will Smith',
 'Robert Downey, Jr.',
 'Sandra Bullock',
 'Kathleen Kennedy',
 'Clint Eastwood',
 'Denzel Washington',
 'Ben Affleck',
 'Vin Diesel',
 'Kevin Feige',
 'Leonardo DiCaprio',
 'John Williams',
 'Steven Spielberg',
 'Tom Hanks',
 'Michael Keaton',
 'Jon Favreau',
 'Kenneth Branagh',
 'Angelina Jolie Pitt',
 'Bradley Cooper',
 'Gwyneth Paltrow',
 'David Heyman',
 'John Lasseter',
 'Alan Silvestri',
 'Scarlett Johansson',
 'Emma Watson',
 'Jim Carrey',
 'Adam Sandler',
 'Matt Damon',
 'Dwayne Johnson',
 'George Clooney',
 'Eddie Redmayne',
 'Ryan Reynolds',
 'Cameron Diaz',
 'Gal Gadot',
 'Samuel L. Jackson',
 'Djimon Hounsou',
 'Brad Pitt',
 "Lupita Nyong'o",
 'Russell Crowe',
 'Julia Roberts',
 'Tom Hardy',
 'Frank Marshall',
 'Harrison Ford',
 'Hugh Jackman',
 'James Wan',
 'Reese Witherspoon',
 'Mark Wahlberg',
 'Johnny Depp',
 'Chris Evans',
 'Mark Ruffalo']

In [4]:
# primary bankability metrics________________________________________________

dirty_bank_list = list(soup.findAll('div', style="font-size:200%;"))
clean_bank_list = []
for line in dirty_bank_list:
    clean_bank_list.append(int(line.text[1:].replace(',','')))
clean_bank_list # test-print

[22537572,
 20593743,
 16602313,
 15694181,
 15416491,
 14466692,
 14189296,
 13770962,
 13610706,
 13144845,
 11501979,
 11486027,
 11175085,
 11091235,
 10979173,
 10851176,
 10802765,
 10786097,
 10482036,
 10223870,
 10080186,
 9864583,
 9679797,
 9675619,
 9467350,
 9083733,
 8899915,
 8838814,
 8679831,
 8678721,
 8459158,
 8454752,
 8415932,
 8402849,
 8180346,
 8130231,
 8100147,
 8081050,
 8023745,
 7982614,
 7924721,
 7891161,
 7876773,
 7781327,
 7728167,
 7663786,
 7581985,
 7566729,
 7527266,
 7352303]

In [5]:
# secondary bankability metrics________________________________________________

sibling_strings = soup.findAll('div', style="font-size:200%;")
movs_per_yr_list = []
bank_per_yr_list = []
bank_per_mov_list = []
for n in sibling_strings:
    string_container = n.nextSibling.nextSibling.get_text()
    # print(string_container, "END \n")  # temp
    # parse through loop and retrieve movies per year for each actor
    divider1 = string_container.find(" movies/year")
    movs_per_yr_list.append(float(string_container[2:divider1]))
    # parse through loop to retrieve bankability per year for each actor
    divider2 = string_container.find("/year", divider1) + divider1
    divider3 = string_container.find("/year", divider2 + divider1)
    bank_per_yr_list.append(int(string_container[divider2 + divider1:divider3].replace(',', '')))
    bank_per_mov_list.append(bank_per_yr_list[-1] / movs_per_yr_list[-1])
# test-prints
print(movs_per_yr_list, len(movs_per_yr_list))
print(bank_per_yr_list, len(bank_per_yr_list))
print(bank_per_mov_list, len(bank_per_mov_list))

[1.1, 1.1, 1.4, 1.0, 1.3, 1.1, 1.2, 1.2, 1.2, 2.2, 1.8, 1.3, 2.9, 2.1, 1.3, 2.0, 1.0, 1.5, 2.5, 1.0, 1.2, 2.2, 1.5, 2.2, 1.2, 1.2, 2.4, 2.6, 2.7, 1.9, 1.0, 2.3, 1.3, 1.1, 4.4, 1.9, 2.8, 1.3, 1.7, 1.4, 1.9, 2.0, 1.3, 2.1, 1.8, 1.3, 2.7, 3.0, 1.9, 2.0] 50
[24791329, 22653117, 23243238, 15694181, 20041438, 15913361, 17027155, 16525154, 16332847, 28918660, 20703562, 14931835, 32407746, 23291593, 14272925, 21702351, 10802765, 16179145, 26205090, 10223870, 12096223, 21702082, 14519696, 21286361, 11360820, 10900479, 21359797, 22980916, 23435545, 16489570, 8459158, 19445930, 10940711, 9243134, 35993523, 15447439, 22680412, 10389806, 13640367, 11175659, 15056970, 15782321, 10239805, 16340787, 13910700, 9962922, 20471359, 22700187, 14301806, 14704605] 50
[22537571.818181816, 20593742.727272727, 16602312.857142858, 15694181.0, 15416490.769230768, 14466691.818181816, 14189295.833333334, 13770961.666666668, 13610705.833333334, 13144845.454545453, 11501978.888888888, 11486026.923076922, 11175084.827

In [6]:
# Primary Roles________________________________________________

films_dict = defaultdict(list)
rank = 1
# print('length = ', len(list(soup.find_all(id="col2mid")))) # temp
for l in list(soup.find_all(id="col2mid")):
    for r in list(re.findall(r'summary">(.*?)<', str(l))):
        if r.find('â') != -1:
            if r[r.find('â')+3] == 's':
                r = r[:r.find('â')]+"'"+r[r.find('â')+3:]
            else:
                r = r[:r.find('â')]+":"+r[r.find('â')+2:]
                # print(r) # temp
        # print('X94! ', r.find('x94')) # r.replace('\\x94','')
        if r not in films_dict[str(rank)]: # check if duplicate, skip if it is
            films_dict[str(rank)].append(r)
    rank +=1
    
# print(rank, films_dict) # WHY?!?!
for rank in films_dict:
    for film in films_dict[rank]:
        print(rank, film)

1 Mission: Impossible:Fallout
1 Mission: Impossible:Rogue Nation
1 Mission: Impossible:Ghost Protocol
1 War of the Worlds
1 Austin Powers in Goldmember
1 Mission: Impossible 2
1 Mission: Impossible III
2 Aladdin
2 Suicide Squad
2 I am Legend
2 Hancock
2 Men in Black 3
2 Hitch
2 The Pursuit of Happyness
2 Annie
2 I, Robot
3 Avengers: Endgame
3 Avengers: Infinity War
3 The Avengers
3 Avengers: Age of Ultron
3 Captain America: Civil War
3 The Judge
3 A Guide to Recognizing Your Saints
4 Minions
4 Gravity
4 The Blind Side
4 Ocean's 8
4 The Heat
4 The Proposal
4 Miss Congeniality
4 Two Weeks Notice
4 Miss Congeniality 2: Armed and Fabulous
4 Hope Floats
5 Star Wars Ep. VII: The Force Awakens
5 Star Wars Ep. VIII: The Last Jedi
5 Rogue One: A Star Wars Story
5 Solo: A Star Wars Story
5 Indiana Jones and the Kingdom of the Crystal Skull
5 Contact
6 The Mule
6 Gran Torino
6 Million Dollar Baby
6 Space Cowboys
6 In the Line of Fire
6 American Sniper
6 Sully
7 The Equalizer 2
7 Safe House
7 T

In [23]:
# put it all together____________________________

main_dict = {}
attributes = {}
if len(clean_actors_list) == len(clean_bank_list):
    for i in range(len(clean_actors_list)):
        attributes['rank'] = i+1
        attributes['bankability'] = clean_bank_list[i]
        attributes['bank per movie'] = bank_per_mov_list[i]
        attributes['roles'] = films_dict[str(i+1)]
        main_dict[clean_actors_list[i]] = attributes.copy()
else:
    raise ValueError
main_dict # test-print
# print(bank_per_mov_list)

# for m in main_dict:
#      print(main_dict[m]['bankability'], main_dict[m]['bank per movie'])

{'Tom Cruise': {'rank': 1,
  'bankability': 22537572,
  'bank per movie': 22537571.818181816,
  'roles': ['Mission: Impossible:\x94Fallout',
   'Mission: Impossible:\x94Rogue Nation',
   'Mission: Impossible:\x94Ghost Protocol',
   'War of the Worlds',
   'Austin Powers in Goldmember',
   'Mission: Impossible 2',
   'Mission: Impossible III']},
 'Will Smith': {'rank': 2,
  'bankability': 20593743,
  'bank per movie': 20593742.727272727,
  'roles': ['Aladdin',
   'Suicide Squad',
   'I am Legend',
   'Hancock',
   'Men in Black 3',
   'Hitch',
   'The Pursuit of Happyness',
   'Annie',
   'I, Robot']},
 'Robert Downey, Jr.': {'rank': 3,
  'bankability': 16602313,
  'bank per movie': 16602312.857142858,
  'roles': ['Avengers: Endgame',
   'Avengers: Infinity War',
   'The Avengers',
   'Avengers: Age of Ultron',
   'Captain America: Civil War',
   'The Judge',
   'A Guide to Recognizing Your Saints']},
 'Sandra Bullock': {'rank': 4,
  'bankability': 15694181,
  'bank per movie': 15694181

In [65]:
filename = "actors_dict"
outfile = open(filename, 'wb')

In [66]:
pickle.dump(main_dict, outfile)
outfile.close()

In [56]:
infile = open(filename,'rb')
new_dict = pickle.load(infile)
infile.close()

In [57]:
# confirm it's loaded accurately
print(new_dict)
print(new_dict==main_dict)
print(type(new_dict))

{'Tom Cruise': {'rank': 1, 'bankability': 22537572, 'bank per movie': 22537571.818181816, 'roles': ['Mission: Impossible:\x94Fallout', 'Mission: Impossible:\x94Rogue Nation', 'Mission: Impossible:\x94Ghost Protocol', 'War of the Worlds', 'Austin Powers in Goldmember', 'Mission: Impossible 2', 'Mission: Impossible III']}, 'Will Smith': {'rank': 2, 'bankability': 20593743, 'bank per movie': 20593742.727272727, 'roles': ['Aladdin', 'Suicide Squad', 'I am Legend', 'Hancock', 'Men in Black 3', 'Hitch', 'The Pursuit of Happyness', 'Annie', 'I, Robot']}, 'Robert Downey, Jr.': {'rank': 3, 'bankability': 16602313, 'bank per movie': 16602312.857142858, 'roles': ['Avengers: Endgame', 'Avengers: Infinity War', 'The Avengers', 'Avengers: Age of Ultron', 'Captain America: Civil War', 'The Judge', 'A Guide to Recognizing Your Saints']}, 'Sandra Bullock': {'rank': 4, 'bankability': 15694181, 'bank per movie': 15694181.0, 'roles': ['Minions', 'Gravity', 'The Blind Side', "Ocean's 8", 'The Heat', 'The P

In [ ]:
infile = open(filename,'rb')
new_dict = pickle.load(infile)
infile.close()

In [41]:
import config
import mysql.connector ## Connect to DB server on AWS

## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = "Movie Project1"
)

cursor = cnx.cursor()
print(cnx)

In [42]:
for actor in main_dict:
    for i in range(len(main_dict[actor]['roles'])):
        # INSERT INDIVIDUALLY
        # print(type(actor), type(main_dict[actor]['bankability']), type(main_dict[actor]['roles'][i]))
        add_actor_line = ("INSERT INTO Actors "
               "(name, bankability, title) "
               "VALUES (%s, %s, %s)")
        data_actor_line = (str(actor), main_dict[actor]['bankability'], main_dict[actor]['roles'][i])
        # Insert new actor line
        cursor.execute(add_actor_line, data_actor_line)

cnx.commit()
        
#         print(main_dict[actor]['rank'],
#           actor,
#           main_dict[actor]['bankability'],
#           main_dict[actor]['roles'][i]
#          )

In [ ]:
# # INSERT INDIVIDUALLY
# add_employee = ("INSERT INTO employees "
#                "(first_name, last_name, hire_date, gender, birth_date) "
#                "VALUES (%s, %s, %s, %s, %s)")

# data_employee = ('Geert', 'Vanderkelen', tomorrow, 'M', date(1977, 6, 14))

In [ ]:
# # INSERT MANY
# data = [
#   ('Jane', date(2005, 2, 12)),
#   ('Joe', date(2006, 5, 23)),
#   ('John', date(2010, 10, 3)),
# ]
# stmt = "INSERT INTO employees (first_name, hire_date) VALUES (%s, %s)"
# cursor.executemany(stmt, data)

In [ ]:
# with open("Bankability.csv", "w") as csvFile:
#     writer = csv.writer(csvFile)
#     writer.writerow(main_dict)
# csvFile.close()